In [1]:
import graphlab

In [ ]:
#data = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train')
#test_data = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/test')

In [14]:
data.save('../gcp-data/MNIST/sframe/train_data')
test_data.save('../gcp-data/MNIST/sframe/test_data')

In [15]:
data = graphlab.SFrame('../gcp-data/MNIST/sframe/train_data')
test_data = graphlab.SFrame('../gcp-data/MNIST/sframe/test_data')

In [16]:
training_data, validation_data = data.random_split(0.8)

In [17]:
training_data['image'] = graphlab.image_analysis.resize(training_data['image'], 28, 28, 1)
validation_data['image'] = graphlab.image_analysis.resize(validation_data['image'], 28, 28, 1)
test_data['image'] = graphlab.image_analysis.resize(test_data['image'], 28, 28, 1)

In [18]:
net = graphlab.deeplearning.create(training_data, target='label')

In [19]:
net.layers

layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[7]: SoftmaxLayer

In [20]:
net.params

{'learning_rate': 0.001, 'momentum': 0.9}

In [21]:
net.layers[4].num_hidden_units = 10

In [22]:
import time
start = time.time()
m = graphlab.neuralnet_classifier.create(training_data, target='label',
                                         network = net,
                                         validation_set=validation_data,
                                         metric=['accuracy', 'recall@2'],
                                         max_iterations=3)
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
  num_hidden_units = 10
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Computing mean image...
PROGRESS: Done computing mean image.
PROGRESS: For Nvidia GPU CUDA support, please pip install http://static.dato.com/files/graphlab-create-1.5.2.gpu.tar.gz
PROGRESS: Creating neuralnet using cpu
PR

In [23]:
pred = m.classify(test_data)
pred

row_id,class,score
0,0,0.999756515026
1,0,0.999999165535
2,0,0.999983668327
3,0,0.999994277954
4,0,0.999995708466
5,0,0.99970895052
6,0,0.999931693077
7,0,0.999987363815
8,0,0.973214387894
9,0,0.999999523163


In [24]:
pred_top2 = m.predict_topk(test_data, k=2)
pred_top2

row_id,class,score
0,0,0.999756515026
0,6,0.000139808907988
1,0,0.999999165535
1,5,6.43732732897e-07
2,0,0.999983668327
2,5,6.18855801804e-06
3,0,0.999994277954
3,5,3.20386197927e-06
4,0,0.999995708466
4,5,2.60134402197e-06


In [25]:
eval_ = m.evaluate(test_data)
eval_

{'accuracy': 0.9681000113487244, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 69
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      4       |        4        |  952  |
 |      9       |        4        |   1   |
 |      2       |        4        |   1   |
 |      6       |        4        |   5   |
 |      5       |        5        |  862  |
 |      0       |        5        |   3   |
 |      9       |        5        |   11  |
 |      3       |        5        |   5   |
 |      7       |        5        |   2   |
 |      6       |        5        |   9   |
 +--------------+-----------------+-------+
 [69 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [19]:
eval_.keys()

['confusion_matrix', 'accuracy']

In [13]:
thang = eval_['confusion_matrix']
thang

target_label,predicted_label,count
2,2,1006
1,2,6
5,2,1
4,2,1
8,2,2
6,2,1
7,2,16
0,2,2
9,2,1
3,2,6
